In [4]:
import argparse
!pip install pydantic
!pip install fastavro

from confluent_kafka import DeserializingConsumer
from confluent_kafka.schema_registry.json_schema import JSONDeserializer
from confluent_kafka.serialization import StringDeserializer


class reqHist(object):
    def __init__(self, candlesize, ticker, from_dt, to_dt):
        self.candlesize = candlesize
        self.ticker = ticker
        self.from_dt = from_dt
        self.to_dt = to_dt

def reqHistDict(user, ctx):
    return dict(candlesize=user.candlesize,
                ticker=user.ticker,
                from_dt=user.from_dt,
               to_dt=user.to_dt)


def delivery_report(err, msg):
    if err is not None:
        print("Delivery failed for data request {}: {}".format(msg.key(), err))
        return
    print('data request {} successfully produced to {} [{}] at offset {}'.format(
        msg.key(), msg.topic(), msg.partition(), msg.offset()))


def main(topic):
#     topic = args.topic

    schema_str = """
    {
      "$schema": "http://json-schema.org/draft-07/schema#",
      "title": "Request Data",
      "description": "Request Data",
      "type": "object",
      "properties": {
        "candlesize": {
          "description": "Size if candlesticks",
          "type": "string"
        },
        "ticker": {
          "description": "String of Ticker",
          "type": "string"
        },
        "from_dt": {
          "description": "date of start",
          "type": "string",
          "format":"date-time"
        },
        "to_dt": {
          "description": "date of end",
          "type": "string",
          "format":"date-time"
          
        }
      },
      "required": [ "candlesize", "ticker", "from_dt","to_dt" ]
    }
    """
    schema_registry_conf = {'url': args.schema_registry}
    schema_registry_client = SchemaRegistryClient(schema_registry_conf)

    json_serializer = JSONSerializer(schema_str, schema_registry_client, user_to_dict)

    producer_conf = {'bootstrap.servers': "kafka1:9092"
                     'key.serializer': StringSerializer('utf_8'),
                     'value.serializer': json_serializer}

    producer = SerializingProducer(producer_conf)

    print("Producing user records to topic {}. ^C to exit.".format(topic))
    while True:
        # Serve on_delivery callbacks from previous calls to produce()
        producer.poll(0.0)
        try:
            pick_candlesize = input("Enter cad: ")
            pick_ticker = input("Enter t: ")
            pick_from =input("Enter from: ")
            pick_to = input("Enter to: ")
            user = reqHist(candlesize=pick_candlesize,
                        ticker=pick_ticker,
                        from_dt=pick_from,
                        to_dt=pick_to)
            producer.produce(topic=topic, key=str(uuid4()), value=user,
                             on_delivery=delivery_report)
        except KeyboardInterrupt:
            break
        except ValueError:
            print("Invalid input, discarding record...")
            continue

    print("\nFlushing records...")
    producer.flush()

main('tester2')
# if __name__ == '__main__':
#     parser = argparse.ArgumentParser(description="SerializingProducer Example")
#     parser.add_argument('-b', dest="bootstrap_servers", required=True,
#                         help="Bootstrap broker(s) (host[:port])")
#     parser.add_argument('-s', dest="schema_registry", required=True,
#                         help="Schema Registry (http(s)://host[:port]")
#     parser.add_argument('-t', dest="topic", default="example_serde_json",
#                         help="Topic name")

#     main(parser.parse_args())

NameError: name 'args' is not defined